In [ ]:
import datetime as dt
import sys
from arch import arch_model
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import yfinance as yf
import tensorflow as tf
from tensorflow import keras
import pandas_datareader as pdr
from datetime import datetime
import seaborn as sns

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

from darts.models import TBATS
from darts.metrics import mape
from sktime.forecasting.base import ForecastingHorizon
from sktime.utils.plotting import plot_series
from sktime.forecasting.model_selection import temporal_train_test_split
from darts import TimeSeries

excel= pd.read_excel("TFG.xlsx", index_col='Date', parse_dates=True)

In [ ]:
excel

# COMPROBACIÓN MODELO LSTM

In [ ]:
# Cargamos los datos de las acciones que queremos analizar y predecir
df = excel[excel.index.year <= 2019]
df

In [ ]:
#Hacemos una gráfica con el histórico de precios
plt.figure(figsize=(16,6))
plt.title('Histórico de precios')
plt.plot(df['Close'], linewidth = 1)
plt.xlabel('Date', fontsize=18)
plt.ylabel('Cierre USD ($)', fontsize=18)
plt.show()

In [ ]:
# Creamos un nuevo DataFrame con únicamente la columna Close
data = df.filter(['Close'])
# Convertimos el dataframe en un numpy array
dataset = data.values

In [ ]:
# Número de filas para entrenar el modelo
training_data_len = int(np.ceil( len(dataset) * .9))

training_data_len

In [ ]:
# Escalamos y normalizamos los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Escalamos y normalizamos los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Creamos el data set para entrenamiento con los datos escalados
train_data = scaled_data[0:int(training_data_len), :]
# Separamos los datos entre x_train y y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
       

In [ ]:
# Convertimos x_train y y_train en numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Cambiamos la forma de los datos
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
# Creamos el modelo LSTM
model = Sequential()

# Añadimos capas LSTM con 50 unidades
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
# Añadimos una capa densa con 25 unidades
model.add(Dense(25))
# Añadir una capa de salida densa con 1 unidad
model.add(Dense(1))

In [ ]:
# Compilamos el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamos el modelo
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# Creamos el conjunto de datos de prueba
test_data = scaled_data[training_data_len - 60: , :]
# Creamos los conjuntos de datos x_test y y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
# Convertimos a un numpy array
x_test = np.array(x_test)

# Cambiamos la forma de los datos
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Obtención de las predicciones
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

In [ ]:
# Calculamos el RMSE
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
print("Root Mean Square Error:",rmse)

In [ ]:
# Calculamos el MAE
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error:", mae)

In [ ]:
# Calculamos el MAPE

# Cambiamos formato
y_test_mape = TimeSeries.from_values(y_test)
predictions_mape = TimeSeries.from_values(predictions)

# Calculamos MAPE
mape_score = mape(y_test_mape, predictions_mape)
mape_score

In [ ]:
# Calculamos el coeficiente de determinación (R cuadrado)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación (R cuadrado):", r2)

In [ ]:
# Graficamos los datos
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,6))
plt.title('NASDAQ')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'], linewidth = 1)
plt.plot(valid[['Close', 'Predictions']], linewidth = 1)
plt.legend(['Datos de entrenamiento', 'Datos de prueba', 'Predicciones'], loc='lower right')
plt.show()

In [ ]:

# Filtrar los datos para el período desde el año 2020 hasta el final
train_2020 = train[train.index.year >= 2019]
valid_2020 = valid[valid.index.year >= 2019]

# Crear la figura y establecer el tamaño
plt.figure(figsize=(16,6))

# Título del gráfico
plt.title('NASDAQ')

# Etiqueta del eje x
plt.xlabel('Date', fontsize=18)

# Etiqueta del eje y
plt.ylabel('Close Price USD ($)', fontsize=18)

# Tramar los datos de entrenamiento
plt.plot(train_2020['Close'], linewidth=1)

# Tramar los datos de prueba y las predicciones
plt.plot(valid_2020[['Close', 'Predictions']], linewidth=1)

# Agregar la leyenda
plt.legend(['Datos de entrenamiento', 'Datos de prueba', 'Predicciones'], loc='lower right')

# Mostrar el gráfico
plt.show()

# MODELO LSTM EN EL COVID-19

In [ ]:
# Cargamos los datos de las acciones que queremos analizar y predecir
df2 = excel[excel.index.year <= 2021]
df2

In [ ]:
# Creamos un nuevo DataFrame con únicamente la columna Close
data = df2.filter(['Close'])
# Convertimos el dataframe en un numpy array
dataset = data.values
len(dataset)

In [ ]:
# Número de filas para entrenar el modelo (cojo 82% de los datos para que llegue hasta 31-12-2019=Covid)
training_data_len = int(np.ceil( len(dataset) * .82259767687434))
training_data_len

In [ ]:
# Escalamos y normalizamos los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Escalamos y normalizamos los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Creamos el data set para entrenamiento con los datos escalados
train_data = scaled_data[0:int(training_data_len), :]
# Separamos los datos entre x_train y y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
       

In [ ]:
# Convertimos x_train y y_train en numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Cambiamos la forma de los datos
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
# Creamos el modelo LSTM
model = Sequential()

# Añadimos capas LSTM con 50 unidades
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
# Añadimos una capa densa con 25 unidades
model.add(Dense(25))
# Añadir una capa de salida densa con 1 unidad
model.add(Dense(1))

In [ ]:
# Compilamos el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamos el modelo
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# Creamos el conjunto de datos de prueba
test_data = scaled_data[training_data_len - 60: , :]
# Creamos los conjuntos de datos x_test y y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
# Convertimos a un numpy array
x_test = np.array(x_test)

# Cambiamos la forma de los datos
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Obtención de las predicciones
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
predictions2 = pd.DataFrame(predictions)
#Para pasarlo a excel: 
predictions2.to_excel('impacto_covid.xlsx', index=False)
predictions2

In [ ]:
# Calculamos el MAE
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error:", mae)

In [ ]:
# Calculamos el coeficiente de determinación (R cuadrado)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación (R cuadrado):", r2)

In [ ]:
# Graficamos los datos
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,6))
plt.title('NASDAQ')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'], linewidth = 1)
plt.plot(valid[['Close', 'Predictions']], linewidth = 1)
plt.legend(['Datos de entrenamiento', 'Datos de prueba', 'Predicciones'], loc='lower right')
plt.show()

In [ ]:
#Graficamos solo el periodo que se predice
#Filtrar los datos para el período desde el año 2021 hasta el final
train_2020 = train[train.index.year >= 2020]
valid_2020 = valid[valid.index.year >= 2020]

# Crear la figura y establecer el tamaño
plt.figure(figsize=(16,6))

# Título del gráfico
plt.title('NASDAQ')

# Etiqueta del eje x
plt.xlabel('Date', fontsize=18)

# Etiqueta del eje y
plt.ylabel('Close Price USD ($)', fontsize=18)

# Tramar los datos de entrenamiento
plt.plot(train_2020['Close'], linewidth=1)

# Tramar los datos de prueba y las predicciones
plt.plot(valid_2020[['Close', 'Predictions']], linewidth=1)

# Agregar la leyenda
plt.legend(['Datos de entrenamiento', 'Datos de prueba', 'Predicciones'], loc='lower right')

# Guardar el Gráfico como jpg
plt.savefig('grafico_Covid.jpg')

# Mostrar el gráfico
plt.show()

# MODELO LSTM EN LA GUERRA DE UCRANIA

In [ ]:
# Cargamos los datos de las acciones que queremos analizar y predecir
df3 = excel[excel.index.year <= 2023]
df3

In [ ]:
# Creamos un nuevo DataFrame con únicamente la columna Close
data = df3.filter(['Close'])
# Convertimos el dataframe en un numpy array
dataset = data.values
len(dataset)

In [ ]:
# Número de filas para entrenar el modelo (cojo 85,6% de los datos para que llegue hasta 1-02-2022-Guerra Ucrania)
training_data_len = int(np.ceil( len(dataset) * .85637342908))
training_data_len

In [ ]:
# Escalamos y normalizamos los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Escalamos y normalizamos los datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)

scaled_data

In [ ]:
# Creamos el data set para entrenamiento con los datos escalados
train_data = scaled_data[0:int(training_data_len), :]
# Separamos los datos entre x_train y y_train
x_train = []
y_train = []

for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])
    if i<= 61:
        print(x_train)
        print(y_train)
        print()
       

In [ ]:
# Convertimos x_train y y_train en numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

# Cambiamos la forma de los datos
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
# Creamos el modelo LSTM
model = Sequential()

# Añadimos capas LSTM con 50 unidades
model.add(LSTM(50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=False))
# Añadimos una capa densa con 25 unidades
model.add(Dense(25))
# Añadir una capa de salida densa con 1 unidad
model.add(Dense(1))

In [ ]:
# Compilamos el modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Entrenamos el modelo
model.fit(x_train, y_train, batch_size=1, epochs=1)

In [ ]:
# Creamos el conjunto de datos de prueba
test_data = scaled_data[training_data_len - 60: , :]
# Creamos los conjuntos de datos x_test y y_test
x_test = []
y_test = dataset[training_data_len:, :]
for i in range(60, len(test_data)):
    x_test.append(test_data[i-60:i, 0])

In [ ]:
# Convertimos a un numpy array
x_test = np.array(x_test)

# Cambiamos la forma de los datos
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Obtención de las predicciones
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
predictions2 = pd.DataFrame(predictions)
# Para pasarlo a excel: 
predictions2.to_excel('impacto_guerra_Ucrania.xlsx', index=False)
predictions2

In [ ]:
# Calculamos el MAE
mae = np.mean(np.abs(predictions - y_test))
print("Mean Absolute Error:", mae)

In [ ]:
# Calculamos el coeficiente de determinación (R cuadrado)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, predictions)
print("Coeficiente de determinación (R cuadrado):", r2)

In [ ]:
# Graficamos los datos
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

plt.figure(figsize=(16,6))
plt.title('Nasdaq')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'], linewidth = 1)
plt.plot(valid[['Close', 'Predictions']], linewidth = 1)
plt.legend(['Datos de entrenamiento', 'Datos de prueba', 'Predicciones'], loc='lower right')
plt.show()

In [ ]:
#Graficamos solo el periodo que se predice
#Filtrar los datos para el período desde el año 2023 hasta el final
train_2020 = train[train.index.year >= 2022]
valid_2020 = valid[valid.index.year >= 2022]

# Crear la figura y establecer el tamaño
plt.figure(figsize=(16,6))

# Título del gráfico
plt.title('NASDAQ')

# Etiqueta del eje x
plt.xlabel('Date', fontsize=18)

# Etiqueta del eje y
plt.ylabel('Close Price USD ($)', fontsize=18)

# Tramar los datos de entrenamiento
plt.plot(train_2020['Close'], linewidth=1)

# Tramar los datos de prueba y las predicciones
plt.plot(valid_2020[['Close', 'Predictions']], linewidth=1)

# Agregar la leyenda
plt.legend(['Datos de entrenamiento', 'Datos de prueba', 'Predicciones'], loc='lower right')

# Guardar el Gráfico como jpg
plt.savefig('grafico_Ucrania.jpg')

# Mostrar el gráfico
plt.show()

# MODELO DE GARCH

In [ ]:
excel=pd.read_excel('TFG.xlsx')
excel

In [ ]:
#Creamos el modelo con p=1 y q=1
model=arch_model(excel['Rentabilidad'], vol='Garch', p=1, q=1)
model_fit=model.fit(update_freq=5)
print(model_fit.summary())

In [ ]:
#Imprimimos los datos de volatilidad diaria con el modelo de Garch
excel['volatility'] = model_fit.conditional_volatility
print(excel[['Rentabilidad', 'volatility']])

In [ ]:
#Lo pasamos a excel
excel.to_excel('modelo_garch.xlsx', index=False)

In [ ]:
#Graficamos los resultados
excel['Fecha'] = pd.to_datetime(excel['Fecha'])
excel.set_index('Fecha', inplace=True)
plt.figure(figsize=(10,6))
plt.plot(excel.index,excel['volatility'])
plt.xlabel("Fecha")
plt.ylabel("Volatilidad")
plt.title('Volatilidad Histórica')
plt.grid(True)
plt.show()
plt.savefig('grafico_garch.jpg')